In [6]:
import pandas as pd

In [7]:
codes = pd.read_csv('cleaned_discharge_summaries.csv').dropna()

In [8]:
keys = ['train', 'dev', 'test']
hadm_ids = {x:list(pd.read_csv('%s_50_hadm_ids.csv'%x, header=None)[0]) for x in keys}
hadm_ids_total = hadm_ids['train'] + hadm_ids['dev'] + hadm_ids['test']

In [9]:
codes = codes[codes['HADM_ID'].isin(hadm_ids_total)].reset_index().drop(['index'], axis=1)
assert len(codes) == len(hadm_ids_total)

In [14]:
texts = {}
labels = {}
for k in keys :
    code_filtered = codes[codes['HADM_ID'].isin(hadm_ids[k])]
    texts[k] = list(code_filtered['TEXT'])
    label = list(code_filtered['ICD9_CODE'])
    label = [x.split(';') for x in label]
    labels[k] = [1 if any(y.startswith('250.00') for y in x) else 0 for x in label]

In [15]:
df_texts = []
df_labels = []
df_exp_split = []

for k in keys :
    df_texts += texts[k]
    df_labels += labels[k]
    df_exp_split += [k]*len(texts[k])
    
df = pd.DataFrame({'text' : df_texts, 'label' : df_labels, 'exp_split' : df_exp_split}) 
df.to_csv('mimic_diabetes_dataset.csv', index=False)

In [17]:
%run "../preprocess_data_BC.py" --data_file mimic_diabetes_dataset.csv --output_file ./vec_diabetes.p \
--word_vectors_type mimic --min_df 5

Vocabulary size :  18888
Found 18884 words in model out of 18888
